In [1]:
import pysam
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import numpy as np
import scipy.stats
import pybedtools as pybed
import sys
import subprocess
#import pymc3 as pm
import re


from utils.SegTools import *
#from utils.plot_tools_eam import *
from utils.vcf_tools_eam0 import *

sns.set_style('ticks')
pd.set_option('display.max_columns', None)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42




df = pd.read_table('../data/annotated_calls/{}_annot_retype.txt'.format("Affy6"))



df_reduce = df[['CHROM', 'BEG_GRCh37', 'END_GRCh37']]
df_reduce.rename(columns={'CHROM':'chrom', 'BEG_GRCh37':'start', 'END_GRCh37':'end'}, inplace=True)

/home/emauryg/anaconda3/envs/bsmn3/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [2]:
##################################
## Data annotation
##################################

## Convert data int SegTools object. 
st = SegTool(df_reduce, sample='SCZ')

## Download cytoband annotations
df_cyto = pd.read_table('utils/cytoBand.txt.gz', names=['chrom','start','end', 'cyto','stain'], compression='gzip')
df_cyto.chrom = df_cyto.chrom.str.replace('chr','')
bed_cyto = pybed.BedTool.from_dataframe(df_cyto[['chrom','start','end','cyto',]])

## Annotate data with cytobands. 
st_cyto = st.annotate(bed_cyto)

def cyto_range(cyto_annot, chrom):
    p = []
    q = []
    for cyto in cyto_annot.split(','):
        if cyto[0] == 'p':
            p.append(float(cyto[1:]))
        else:
            q.append(float(cyto[1:]))
    if p:
        start = "p{}".format(p[-1])
        if q:
            end = "q{}".format(q[-1])
        else:
            end = "p{}".format(p[0])
    else:
        start = "q{}".format(q[0])
        end = "q{}".format(q[-1])
    return "{}{}-{}".format(chrom,start,end)
    

cyto_lst = [cyto_range(row.annotation, row.chrom) for i, row in st_cyto.seg.iterrows()]

cyto_lst2 = [",".join(["{}{}".format(row.chrom, s) for s in row.annotation.split(',')]) for i, row in st_cyto.seg.iterrows()]

df['CYTOBAND'] = cyto_lst

df_annot = df_reduce.copy()
df_annot['cyto'] = cyto_lst2

## Annotate with gene names
st = SegTool(df_reduce, sample='SCZ')
f_genes= "utils/hg19_genes.bed"
df_genes = pd.read_table(f_genes, names = ['chrom','start','end','gene'])
df_genes.chrom = df_genes.chrom.str.replace("chr","")
df_genes.start = df_genes.start.astype(int)
df_genes.end = df_genes.end.astype(int)

bed_genes = pybed.BedTool.from_dataframe(df_genes)

st_an = st.annotate(bed_genes)
df_annot['genes'] = st_an.seg['annotation']
gene_lst = [",".join(sorted(set([an.split('-')[0] for an in row.genes.split(',')]))) for i, row in df_annot.iterrows()]
df['GENES'] = gene_lst


/mnt/c/Users/emaur/Dropbox (MIT)/Walsh Lab/SCZ_sCNV/SCZ_sCNV_paper_repo/scripts/utils/SegTools.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.seg['length'] = self.seg.end - self.seg.start
/mnt/c/Users/emaur/Dropbox (MIT)/Walsh Lab/SCZ_sCNV/SCZ_sCNV_paper_repo/scripts/utils/SegTools.py:298: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[name] = annotations


In [5]:
#### Gene annotations
df.reset_index(inplace=True,drop=True)

ngenes = np.zeros(len(df))

for i, row in df.iterrows():
  genes_affected = row.GENES.split(",")
  if '' in genes_affected:
    pass
  else:
    num_genes = len(genes_affected)
    ngenes[i] = num_genes
  

df['number_genes'] = ngenes
df["log_number_genes"] = np.log(df.number_genes + 1)

df_annotated = df.copy()
df_annotated.reset_index(inplace=True, drop=True)
def gene_count(mcnv_table, gene_set):
    # input mcnv_table is the table that you want annotate, note it assumes there is a string column GENES with comma separated gene names
    # input gene_set a vector with the gene names of a set you want to count/label
    # input set_label is the label of the set e.g. "high pLI genes"
    num_burden_genes = np.zeros(len(mcnv_table))
    for i, row in mcnv_table.iterrows():
        genes_affected = row.GENES.split(",")
        if not '' in genes_affected:
            try:
                num_burden_genes[i] = sum(pd.Series(genes_affected).isin(gene_set.genes.values))
            except KeyError as error:
                print(error)
                print("The Gene missing was in row ",i)
                continue
    return(num_burden_genes)


## High pLI annotation

snv_constraint_db = pd.read_csv("../data/metadata/fordist_cleaned_nonpsych_z_pli_rec_null_data.txt",sep="\t")
pli_genes = snv_constraint_db.gene[snv_constraint_db.pLI >= 0.90].to_frame()
pli_genes.columns = ["genes"]

df_annotated["high_pLI_genes"] = gene_count(df_annotated, pli_genes)

## Brain expression
gtex_expression_matrix = pd.read_csv('../data/metadata/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz',compression="gzip",delimiter="\t",skiprows=2)
stacked_expression = pd.melt(gtex_expression_matrix, id_vars=["Name","Description"], value_vars=np.array(gtex_expression_matrix.columns.astype("str")[2:]))
stacked_expression["isBrain"] = 0
stacked_expression.isBrain[stacked_expression.variable.str.contains("Brain")] = 1

## calculate the fold change of brain expression
stacked_expression["brain_FC"] = 0
Nb = sum(gtex_expression_matrix.columns.str.contains("Brain"))
N =  sum(gtex_expression_matrix.columns.str.contains("Brain")) + sum(gtex_expression_matrix.columns.str.contains("Brain") == False)
brain_expression = stacked_expression[(stacked_expression.isBrain)==1].sort_values(["value"],ascending=False).reset_index()

border_gene = len(brain_expression)*0.20

top20_genes = brain_expression.Description[0:(int(border_gene)-1)]
top20_genes = top20_genes[top20_genes.str.contains("MT-")==False].to_frame()
top20_genes.columns = ["genes"]
top20_genes = top20_genes.drop_duplicates(subset="genes")

df_annotated["brain_genes"] = gene_count(df_annotated, top20_genes)

## Synaptome annotation
synaptome_genes = pd.read_csv("../data/metadata/synaptome.txt",header=None,names=["genes"])
synaptome_genes.genes = synaptome_genes.genes.astype("str")

df_annotated["synaptic_genes"] = gene_count(df_annotated, synaptome_genes)

## ABC gene annotation
abc_genes = pd.read_csv("../data/metadata/abc_genes.txt", header=None, names=["genes"])
abc_genes.genes = abc_genes.astype("str")
df_annotated["abc_genes"] = gene_count(df_annotated, abc_genes)


/home/emauryg/anaconda3/envs/bsmn3/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
df_annotated.to_csv("../data/{}_nofilter_mCNVs.csv".format("Affy6"), index=False, sep=",")
